## 1.处理数据

In [1]:
import os
import requests
import zipfile
import tarfile
import hashlib
import plotly.graph_objs as go
from d2l import mxnet as d2l
from mxnet import gluon, autograd, init, np, npx
from mxnet.gluon import nn
import pandas as pd
%matplotlib inline
npx.set_np()
ctx = npx.gpu() if (npx.num_gpus()) > 0 else mx.cpu()

OSError: libcuda.so.1: cannot open shared object file: No such file or directory

### 1.1 获取数据
可以在kaggle官网下载：https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

训练数据集包含1460个示例，80个特征和1个标签，而测试数据包含1459个示例和80个特征。

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_data.columns

查看一下特征的类型， object类型的比较多，也就是说大部分是category类型

In [ ]:
train_data.dtypes.value_counts().plot('bar')

通过直方图，看一下房价的分布，符合泊松分布

In [ ]:
train_data.SalePrice.plot.hist()

看一下数据，在每个示例中，第一个功能都是ID。它不携带任何用于预测目的的信息。因此，我们在将数据输入模型之前将其从数据集中删除。

In [ ]:
train_data.head(4)

将训练数据和测试数据合并一起进行预处理

In [ ]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

### 1.3 数据预处理
我们有各种各样的数据类型。在开始建模之前，我们需要对数据进行预处理。首先处理数值类型数据：
+ 我们应用试探法，将所有缺失值替换为相应特征的均值。
+ 我们对连续数值的特征做标准化（standardization）。
+ 我们将数据标准化有两个原因。第一，这样更方便计算。第二，不知道哪些特征将是相关的，要对所有的特征进行同样比率的惩罚

In [ ]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))

+ 为了不影响平均值和标准差，优先进行标准化，这时所有特征的均值都是0,将缺失值替换为均值，也就是0

In [ ]:
all_features[numeric_features] = all_features[numeric_features].fillna(0)

+ 离散型数据进行onehot处理， dummy_na=True将缺失值也当作合法的特征值并为其创建指示特征
+ onehot处理之后，特征的数量从79个增加到331个

In [ ]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

+ 可以从该pandas格式中提取NumPy 格式，并将其转换为张量表示形式以进行训练。

In [ ]:
n_train = train_data.shape[0]
train_features = np.array(all_features[:n_train].values, dtype=np.float32, ctx=ctx)
test_features = np.array(all_features[n_train:].values, dtype=np.float32, ctx=ctx)
train_labels = np.array(train_data.SalePrice.values.reshape(-1, 1), dtype=np.float32, ctx=ctx)

## 2.训练
我们使用一个基本的线性回归模型和平方损失函数来训练模型。对于房价和股票价格，我们关心的是相对数量而不是绝对数量。更加关心相对误差，因此使用对数均方根误差，这也是比赛用来评估提交质量的官方错误度量。公式如下：
$$
\sqrt{\frac{1}{n}\sum_{i=1}^n\left(\log y_i -\log \hat{y}_i\right)^2}
$$

In [ ]:
loss = gluon.loss.L2Loss()

def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize(ctx = ctx)
    return net

def log_rmse(net, features, labels):
    # 将小于1的值设成1，使得取对数时数值更稳定
    clipped_preds = np.clip(net(features), 1, float('inf'))
    return np.sqrt(2 * loss(np.log(clipped_preds), np.log(labels)).mean())

下面的训练函数与本章中前几节的不同在于使用了Adam优化算法。相对之前使用的小批量随机梯度下降，它对学习率相对不那么敏感。

In [ ]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用了Adam优化算法
    trainer = gluon.Trainer(net.collect_params(), 'adam', {
        'learning_rate': learning_rate, 'wd': weight_decay})
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

## 3. K折交叉验证
它将被用来选择模型设计并调节超参数。下面实现了一个函数，它返回第i折交叉验证时所需要的训练和验证数据。

In [ ]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_valid, y_valid = X[i*fold_size:(i+1)*fold_size], y[i*fold_size:(i+1)*fold_size]
    X_train, y_train = np.concatenate([X[:i*fold_size], X[(i+1)*fold_size:]], 0), np.concatenate([y[:i*fold_size], y[(i+1)*fold_size:]], 0)
    return X_train, y_train, X_valid, y_valid

In [ ]:
def draw_graph(result, yaxis_range=[-3,2], names = ['train', 'valid'], yaxis_type='log'):
    data = []
    colors = ['aquamarine', 'hotpink']
    for i, info in enumerate(result[1:]):
        trace = go.Scatter(
            x = result[0],
            y = info,
            mode = 'lines+markers',
            name = names[i],
            marker = {
                'color':colors[i],
            },
        )
        data.append(trace)
        layout = go.Layout(yaxis = {
            'type':yaxis_type,
            'range':yaxis_range,
            'title_text': 'rmse'
        })
    fig = go.Figure(data = data, layout=layout)
    fig.update_xaxes(title_text = "epoch")
    fig.update_layout(width=800, height=480)
    fig.show()

In [ ]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            draw_graph([list(range(1, num_epochs+1)),train_ls, valid_ls], [-1,1])
        print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, '
              f'valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [ ]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-fold validation: avg train log rmse: {float(train_l):f}, '
      f'avg valid log rmse: {float(valid_l):f}')

## 4.Kaggle上提交预测
下面定义预测函数。在预测之前，我们会使用完整的训练数据集来重新训练模型，并将预测结果存成提交所需要的格式。

In [ ]:
def train_and_pred(train_features, test_feature, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, weight_decay, batch_size)
    draw_graph([list(range(1, num_epochs+1)),train_ls], [-1,1])
    print(f'train log rmse {float(train_ls[-1]):f}')
    # 测试数据通过训练好的模型获得预测值
    preds = d2l.numpy(net(test_features))
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    # 将预测值导出文件上传到kaggle上
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('submission.csv', index=False)

In [ ]:
train_and_pred(train_features, test_features, train_labels, test_data,
               num_epochs, lr, weight_decay, batch_size)